In [322]:
import pandas as pd
import numpy as np

In [323]:
df = pd.read_csv("/Users/massimilianoarca/Documents/5th Year/2nd Semester/Internet Of Things/Project/Dataset - prova.csv")
df = pd.DataFrame(df)

df.head()

,No.,Time,Time since previous frame,Source,Destination,Time to Live,Data Length,Frame Length,Protocol
0,1,0.000000,0.000000,192.168.1.226,188.153.150.68,64,130.0,172,UDP
1,2,0.008723,0.008723,188.153.150.68,192.168.1.226,52,843.0,885,UDP
2,3,0.008725,0.000002,188.153.150.68,192.168.1.226,52,835.0,877,UDP
3,4,0.008726,0.000001,188.153.150.68,192.168.1.226,52,903.0,945,UDP
4,5,0.008726,0.000000,188.153.150.68,192.168.1.226,52,903.0,945,UDP


In [324]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['Time since previous frame'] = pd.to_datetime(df['Time since previous frame'], unit='s')

df = df.drop(columns=['No.'])

In [325]:
dst_address = "188.153.150.68"

In [326]:
df_complete = df.resample('0.5S', on='Time').agg(
    {'Time to Live' : np.mean,'Time since previous frame' : np.mean,
     'Data Length' : np.mean, 'Frame Length' : np.mean})

In [327]:
df_complete.head()

,Time to Live,Time since previous frame,Data Length,Frame Length
Time,,,,
1970-01-01 00:00:00.000,58.582677,1970-01-01 00:00:00.001290110,847.868074,886.204724
1970-01-01 00:00:00.500,58.155620,1970-01-01 00:00:00.003429766,862.419825,895.538905
1970-01-01 00:00:01.000,57.883871,1970-01-01 00:00:00.001628906,837.916129,879.916129
1970-01-01 00:00:01.500,58.707865,1970-01-01 00:00:00.004532853,811.333333,837.168539
1970-01-01 00:00:02.000,58.382271,1970-01-01 00:00:00.003002792,818.714286,855.875346


In [328]:
df_inbound = df.loc[df['Source'] == dst_address]
df_outbound = df.loc[df['Destination'] == dst_address]

In [329]:
df_inbound.head()

,Time,Time since previous frame,Source,Destination,Time to Live,Data Length,Frame Length,Protocol
1,1970-01-01 00:00:00.008723,1970-01-01 00:00:00.008723,188.153.150.68,192.168.1.226,52,843.0,885,UDP
2,1970-01-01 00:00:00.008725,1970-01-01 00:00:00.000002,188.153.150.68,192.168.1.226,52,835.0,877,UDP
3,1970-01-01 00:00:00.008726,1970-01-01 00:00:00.000001,188.153.150.68,192.168.1.226,52,903.0,945,UDP
4,1970-01-01 00:00:00.008726,1970-01-01 00:00:00.000000,188.153.150.68,192.168.1.226,52,903.0,945,UDP
5,1970-01-01 00:00:00.012023,1970-01-01 00:00:00.003297,188.153.150.68,192.168.1.226,52,903.0,945,UDP


In [330]:
df_outbound.head()

,Time,Time since previous frame,Source,Destination,Time to Live,Data Length,Frame Length,Protocol
0,1970-01-01 00:00:00.000000,1970-01-01 00:00:00.000000,192.168.1.226,188.153.150.68,64,130.0,172,UDP
12,1970-01-01 00:00:00.019632,1970-01-01 00:00:00.000336,192.168.1.226,188.153.150.68,64,126.0,168,UDP
13,1970-01-01 00:00:00.024108,1970-01-01 00:00:00.004476,192.168.1.226,188.153.150.68,64,1175.0,1217,UDP
14,1970-01-01 00:00:00.024127,1970-01-01 00:00:00.000019,192.168.1.226,188.153.150.68,64,1087.0,1129,UDP
15,1970-01-01 00:00:00.024129,1970-01-01 00:00:00.000002,192.168.1.226,188.153.150.68,64,830.0,872,UDP


In [331]:
df_inbound = df_inbound.drop(columns=['Destination'])
df_outbound = df_outbound.drop(columns=['Source'])

In [332]:
df_inbound = df_inbound.resample('0.5S', on='Time').agg(
    {'Source' : 'count', 'Time to Live' : np.mean,'Time since previous frame' : np.mean,
     'Data Length' : np.mean, 'Frame Length' : np.mean})

df_inbound = df_inbound.rename(columns={'Source' : 'Inbound Count'})

In [333]:
df_outbound = df_outbound.resample('0.5S', on='Time').agg(
    {'Destination' : 'count', 'Time to Live' : np.mean,'Time since previous frame' : np.mean,
     'Data Length' : np.mean, 'Frame Length' : np.mean})

df_outbound = df_outbound.rename(columns={'Destination' : 'Outbound Count'})

In [334]:
inbound_count = df_inbound["Inbound Count"]
outbound_count = df_outbound["Outbound Count"]

df_complete = df_complete.join(inbound_count)
df_complete = df_complete.join(outbound_count)

In [335]:
df_complete.head()

,Time to Live,Time since previous frame,Data Length,Frame Length,Inbound Count,Outbound Count
Time,,,,,,
1970-01-01 00:00:00.000,58.582677,1970-01-01 00:00:00.001290110,847.868074,886.204724,172,209
1970-01-01 00:00:00.500,58.155620,1970-01-01 00:00:00.003429766,862.419825,895.538905,169,178
1970-01-01 00:00:01.000,57.883871,1970-01-01 00:00:00.001628906,837.916129,879.916129,158,152
1970-01-01 00:00:01.500,58.707865,1970-01-01 00:00:00.004532853,811.333333,837.168539,157,199
1970-01-01 00:00:02.000,58.382271,1970-01-01 00:00:00.003002792,818.714286,855.875346,169,192


In [336]:
df_complete.reset_index(inplace=True)

df_complete.head()

,Time,Time to Live,Time since previous frame,Data Length,Frame Length,Inbound Count,Outbound Count
0,1970-01-01 00:00:00.000,58.582677,1970-01-01 00:00:00.001290110,847.868074,886.204724,172,209
1,1970-01-01 00:00:00.500,58.155620,1970-01-01 00:00:00.003429766,862.419825,895.538905,169,178
2,1970-01-01 00:00:01.000,57.883871,1970-01-01 00:00:00.001628906,837.916129,879.916129,158,152
3,1970-01-01 00:00:01.500,58.707865,1970-01-01 00:00:00.004532853,811.333333,837.168539,157,199
4,1970-01-01 00:00:02.000,58.382271,1970-01-01 00:00:00.003002792,818.714286,855.875346,169,192


In [337]:
df_complete['Time'] = df_complete['Time'].dt.time
df_complete['Time since previous frame'] = df_complete['Time since previous frame'].dt.time

In [338]:
df_complete.head()

,Time,Time to Live,Time since previous frame,Data Length,Frame Length,Inbound Count,Outbound Count
0,00:00:00,58.582677,00:00:00.001290,847.868074,886.204724,172,209
1,00:00:00.500000,58.155620,00:00:00.003429,862.419825,895.538905,169,178
2,00:00:01,57.883871,00:00:00.001628,837.916129,879.916129,158,152
3,00:00:01.500000,58.707865,00:00:00.004532,811.333333,837.168539,157,199
4,00:00:02,58.382271,00:00:00.003002,818.714286,855.875346,169,192
